# Introspecting models

To help investigators work with models and simulations, [BioSimulators-utils](https://github.com/biosimulators/Biosimulators_utils) provides a method, `get_parameters_variables_outputs_for_simulation` for introspecting model/simulation files. This method can be helpful for programmatically constructing SED-ML files and COMBINE archives from other formats.

This method can extract several types of information about model/simulation files:
* Inputs to simulations: Boolean and numeric-valued attributes of models/simulations such as for constants and initial conditions.
* Outputs of simulations: Possible variables which can be observed such as values of objectives, concentrations of species, fluxes of reactions, and sizes of compartments.
* Simulations: Settings for simulations such as start and stop times, algorithms, and algorithm parameters.
* Plots: Settings for plots of simulation results, such as the obserables that should be painted on each curve of a 2D line plot.

This method currently supports the following formats:
* [BioNetGen Language (BNGL)](https://www.bionetgen.org/)
* [CellML](https://www.cellml.org/)
* [GINsim Markup Language (GINML, ZGINML)](http://ginsim.org/)
* [NeuroML](https://neuroml.org/)/[Low Entropy Model Specification (LEMS)](https://lems.github.io/LEMS/)
* [Resource Balance Analysis (RBA)](https://sysbioinra.github.io/RBApy/)
* [Smoldyn](http://www.smoldyn.org/)
* [Systems Biology Markup Language (SBML)](http://sbml.org)
* [SBML flux balance constraints (FBC) package](http://sbml.org/Documents/Specifications/SBML_Level_3/Packages/fbc)
* [SBML qualitative models (qual) package](http://sbml.org/Documents/Specifications/SBML_Level_3/Packages/qual)
* [SBML Mass Action Stoichiometric Simulation (MASS) schema](https://masspy.readthedocs.io/en/stable/tutorials/reading_writing_models.html)
* [XPP ODE format](http://www.math.pitt.edu/~bard/xpp/help/xppodes.html)

This tutorial illustrates how to use BioSimulators utils to introspect a model.

## 1. Install BioSimulators-utils with options for the model formats that you would like to introspect

For example, install BioSimulators-utils with the `sbml` option to enable introspection of models encoded in SBML. More information about the available installation options is available at [https://docs.biosimulators.org/](https://docs.biosimulators.org/Biosimulators_utils/).

In [1]:
!pip install biosimulators-utils[sbml]

Invalid -W option ignored: invalid module name: 'biosimulators_utils.warnings'
Defaulting to user installation because normal site-packages is not writeable


## 2. Import `get_parameters_variables_outputs_for_simulation`

In [2]:
from biosimulators_utils.sedml.model_utils import get_parameters_variables_outputs_for_simulation

## 3. Import `ModelLanguage` to describe the format that should be introspected

In [3]:
from biosimulators_utils.sedml.data_model import ModelLanguage

## 4. Import an additional class to describe the default type of simulation that should be introspected from the model file if the file does not describe any simulations

In [4]:
from biosimulators_utils.sedml.data_model import UniformTimeCourseSimulation

## 5. Execute `get_parameters_variables_outputs_for_simulation` on a model file

Execute the following code to introspect the Systems Biology Markup Language (SBML) file for the [Ciliberto et al. morphogenesis checkpoint model](../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml).

In [5]:
inputs, simulations, outputs, plots = get_parameters_variables_outputs_for_simulation(
    model_filename='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
    model_language=ModelLanguage.SBML,
    simulation_type=UniformTimeCourseSimulation)

The first output, `inputs`, describes the inputs to simulations of the model. This captures all of the attributes of a simulation that could be modified. This output is a list of instances of `biosimulators_utils.sedml.data_model.ModelAttributeChange`. Each instance captures a suggested id and name for a change for for the corresponding model component, the address of the component within the model, and its default value encoded into a string.

In [6]:
import yaml
print(yaml.dump({input.id: {'target': input.target, 'default': input.new_value} for input in inputs}))

init_conc_species_BE:
  default: '2.429618e-4'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='BE']/@initialConcentration
init_conc_species_Cdc20:
  default: '1.1722378'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cdc20']/@initialConcentration
init_conc_species_Cdc20a:
  default: '1.4384692'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cdc20a']/@initialConcentration
init_conc_species_Cdh1:
  default: '0.99263656'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cdh1']/@initialConcentration
init_conc_species_Clb:
  default: '0.18453673'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Clb']/@initialConcentration
init_conc_species_Cln:
  default: '0.053600963'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cln']/@initialConcentration
init_conc_species_IE:
  default: '0.52220768'
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='IE']/@initi

The second output, `simulations`, describes the simulations specified in the model file. This output is a list of instances of `biosimulators_utils.sedml.data_model.Simulation`. If no simulations were described in the file, `get_parameters_variables_outputs_for_simulation` returns an array with a single instance of the default type of simulation with a recommended simulation algorithm.

In [7]:
for i_simulation, simulation in enumerate(simulations):
    print(f'Simulation {i_simulation + 1}')
    print(f' Type: {simulation.__class__.__name__}')
    print(f' Initial time: {simulation.initial_time}')
    print(f' Output start time: {simulation.output_start_time}')
    print(f' Output end time: {simulation.output_end_time}')
    print(f' Number of steps: {simulation.number_of_steps}')
    print(f' Algorithm: {simulation.algorithm.kisao_id}')

Simulation 1
 Type: UniformTimeCourseSimulation
 Initial time: 0.0
 Output start time: 0.0
 Output end time: 1.0
 Number of steps: 10
 Algorithm: KISAO_0000019


The third output, `outputs`, describes the possible outputs that could be recorded from simulations of the model. This output is a list of instances of `biosimulators_utils.sedml.data_model.Variable`. Each instance captures a suggested id and name for and observable for the corresponding model component and its address within the model.

In [8]:
import yaml
print(yaml.dump({output.id: {'target': output.target, 'symbol': output.symbol} for output in outputs}))

dynamics_species_BE:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='BE']
dynamics_species_Cdc20:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cdc20']
dynamics_species_Cdc20a:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cdc20a']
dynamics_species_Cdh1:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cdh1']
dynamics_species_Clb:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Clb']
dynamics_species_Cln:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Cln']
dynamics_species_IE:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='IE']
dynamics_species_Mcm:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:species[@id='Mcm']
dynamics_species_Mih1a:
  symbol: null
  target: /sbml:sbml/sbml:model/sbml:listOfSpecies/sbml:s

The fourth output, `plots`, describes the plots specified in the model file. This output is a list of instances of `biosimulators_utils.sedml.data_model.Plot`.

In [9]:
import yaml
print(yaml.dump([plot.id for plot in plots]))

[]



<div class="alert alert-block alert-info">
    All model formats contain information about parameters and variables. However, most model formats, such as SBML, do not contain any information about simulations and plots.
</div>

## 6. Getting the default value of each input in its native data type

By default, `get_parameters_variables_outputs_for_simulation` returns the default value of each simulation input as a string because the [Simulation Experiment Description Model Language (SBML)](http://sed-ml.org/) uses strings to describe the values of model changes.

As illustrated below, the `native_data_types` option can be used to return the default value of each simulation input in its native data type (e.g., `bool`, `float`, `int`).

In [10]:
native_inputs, _, _, _ = get_parameters_variables_outputs_for_simulation(
    model_filename='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
    model_language=ModelLanguage.SBML,
    simulation_type=UniformTimeCourseSimulation,
    native_data_types=True)

In [11]:
print(f'SED-ML data type: {inputs[0].new_value.__class__}')
print(f'Native data type: {native_inputs[0].new_value.__class__}')

SED-ML data type: <class 'str'>
Native data type: <class 'float'>


## 7. Getting the ids and names of the model components corresponding to each simulation input and output

By default, `get_parameters_variables_outputs_for_simulation` returns suggested ids and names for model changes and observables for inputs and outputs. As illustrated below, the `native_ids` option can be used to retrieve the id and name of each corresponding model element.

In [12]:
native_inputs, _, native_outputs, _ = get_parameters_variables_outputs_for_simulation(
    model_filename='../_data/Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.xml',
    model_language=ModelLanguage.SBML,
    simulation_type=UniformTimeCourseSimulation,
    native_ids=True)

In [13]:
print(f'Id of a model component that can be changed: {native_inputs[0].id}')
print(f'Suggested id for a change to this component: {inputs[0].id}')

Id of a model component that can be changed: Trim
Suggested id for a change to this component: init_conc_species_Trim


In [14]:
print(f'Id of a model component that can be observed: {native_outputs[1].id}')
print(f'Suggested id for an observable for the model component: {outputs[1].id}')

Id of a model component that can be observed: Trim
Suggested id for an observable for the model component: dynamics_species_Trim


<div class="alert alert-block alert-info">Note, implicit outputs that are not explicitly specified in model files do not have ids.</div>